In [1]:
from keras.preprocessing import image
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
model = keras.models.load_model('masknet.h5')

In [4]:
face_model = cv2.CascadeClassifier('haarcascade-frontal-facedefault.xml')

In [7]:
mask_label = {0:'MASK',1:'NO MASK'}

cap = cv2.VideoCapture(0)  
while(True):
    ret, frame = cap.read() 
    img = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)
    faces = face_model.detectMultiScale(img)

    label = [0 for i in range(len(faces))]
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        crop = frame[y:y+h,x:x+w]
        crop = cv2.resize(crop,(128,128))
        crop = np.reshape(crop,[1,128,128,3])/255.0
        mask_result = model.predict(crop)
        cv2.putText(frame,mask_label[mask_result.argmax()],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,2)
    cv2.imshow('Frame',frame)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()